# Generate configuration file

Generate hardware and software configuration files
* **Hardware configuration file**: configure the hardware (neurons, synapses, ion channels, monitoring)
* **Software configuration file**: configure the C++ application (monitoring, stimulation, control)

In [2]:
%load_ext autoreload
%autoreload 2
from configuration.gen_config import gen_config, NetwConfParams

# Configuration file 
CONFIG_NAME = "singleProva2"
SAVE_PATH   = "export/"

# Stimulation
netw_conf = NetwConfParams()
netw_conf.model                     = "single" # "custom", "single", "connectoid"
netw_conf.emulation_time_s          = 5
netw_conf.en_step_stim              = False
netw_conf.step_stim_delay_ms        = 0
netw_conf.step_stim_duration_ms     = 0
netw_conf.local_save_path           = "/home/ubuntu/bioemus/data/"
netw_conf.en_randomize_hh_params    = True
netw_conf.val_randomize_hh_params   = 0.10
netw_conf.org_wsyninh               = 1.0
netw_conf.org_wsynexc               = 0.22
netw_conf.org_pcon_in               = 0.08
netw_conf.org_pcon_out              = 0.02
netw_conf.org_wsyn_in               = 1.0
netw_conf.org_wsyn_out              = 1.0
netw_conf.org_inh_ratio             = 0.2


[hwconfig, swconfig] = gen_config(CONFIG_NAME, netw_conf, SAVE_PATH)

Software configuration file saved at: export/swconfig_singleProva2.json
Hardware configuration file saved at: export/hwconfig_singleProva2.txt


# Emulate configuration

* Emulate using the configuration files generated

In [1]:
import numpy as np
from emulation.emulate_config import emulate_config

NEURON_LIST   = [i for i in range(4)]
FPGA_EMU      = False
STORE_CONTEXT = False

exact_emu = emulate_config(hwconfig, swconfig, NEURON_LIST, False, STORE_CONTEXT, dtype=np.float64)

if FPGA_EMU:
    fpga_emu = emulate_config(hwconfig, swconfig, NEURON_LIST, True,  STORE_CONTEXT, dtype=np.float32)

NameError: name 'hwconfig' is not defined

* Plot emulation

In [ ]:
%matplotlib qt
from emulation.plots.SnnPlotter import *

NEURON_TO_DISPLAY       = 0
NEURON_LIST_TO_DISPLAY  = [0,1,2,3]

exact_plotter = SnnPlotter(exact_emu)
# exact_plotter.plotIonRates()
# exact_plotter.plotSynRates()
# exact_plotter.plotIonChanStates(NEURON_TO_DISPLAY)
# exact_plotter.plotCurrents(NEURON_TO_DISPLAY)
exact_plotter.plotVmem(NEURON_LIST_TO_DISPLAY, plot_type='all')
# exact_plotter.plotVmem(NEURON_LIST_TO_DISPLAY, plot_type='comp')
# exact_plotter.plotVmem(NEURON_LIST_TO_DISPLAY, plot_type='subplot')
# exact_plotter.plotRaster()

# Monitoring

* Monitor spikes

In [ ]:
import monitoring.spkmon.main as spike_monitor
spike_monitor.main()

* Monitor waves

In [ ]:
# Monitor one wave with GUI
# import monitoring.vmon.main as vmem_monitor
# vmem_monitor.main()

# Monitor multiple waves
from monitoring.waves.waves_mon import waves_mon

TARGET_IP           = "192.168.137.16"
TARGET_PORT         = "5558"
NB_CHANNELS         = 16
NB_DT_PER_TRANSFER  = 190
WINDOW_SIZE_S       = 1
REFRESH_TIME_S      = 0.1

waves_mon(f"tcp//{TARGET_IP}:{TARGET_PORT}", NB_CHANNELS, NB_DT_PER_TRANSFER, WINDOW_SIZE_S, REFRESH_TIME_S)

* External stimulation

In [ ]:
import monitoring.ext_stim.main as external_stim
external_stim.main()

# Analysis

* Raster

In [ ]:
%matplotlib qt
from analysis.draw_raster import draw_raster
from analysis.extract_spikes import extract_spikes
from analysis.spike_analysis import spike_analysis
from analysis.burst_analysis import burst_analysis

DIRPATH        = "data/"
RASTER_LIST    = [f"single{i}" for i in range(1,4+1)]
RASTER_LIST    = f"single5"
REC_DURATION_S = 5
SAVE_FIGS      = False

draw_raster(DIRPATH, RASTER_LIST, SAVE_FIGS)

tstamp_list = extract_spikes(DIRPATH, RASTER_LIST, REC_DURATION_S)
spike_analysis(RASTER_LIST, tstamp_list, REC_DURATION_S)
burst_analysis(RASTER_LIST, tstamp_list)

* Waveforms

In [ ]:
%matplotlib qt
from analysis.draw_waves import draw_waves

DIRPATH        = "data/"
WAVE_LIST      = ["test1", "test2"]
PLOT_TIME_S    = 0.1
SEL_NRN        = [i for i in range(2)] # up to MAX_NB_NEURONS_WAVES_MON
SAVE_FIGS      = False

draw_waves(DIRPATH, WAVE_LIST, PLOT_TIME_S, SEL_NRN)
